# Strategy Design Pattern

Super havalı bir projen olduğunu düşünelim. Mesela evindeki malzemelere göre yemek tarifi veren  bir uygulama. 

Ancak bir süre sonra kullanıclar süre sınırından bahsetmeye başlıyor. Tek kısıt evdeki malzemeler değil bazen süre sınırı da olabiliyor. Bu yüzden artık süre limitine göre de filitreleme yapan bir özellik ekliyorsun.  


Kullanıcılar çok mutlu ama bir süre sonra yeni bir istek daha geliyor. Tarifleri imkanlara gre filitreleme. Herkesin çelik bir wok ya fa döküm tava olmayabilir.  

Git gide özellikler artıyor. Yeni özellikler kullanıcıları çok mutlu etsede teknik ekip düzenli olarak ağlamaktan ciddi su kaybı yaşamaya başladı. İşler karıştıkça teknik problemler artacak.  

Eğer bir geliştirii olacak çalışacaksan mutlaka beklenmedik özellikler eklemek zorunda kalacaksın.

Bunun için tek çözüm beklenmedik özelliklere hazır temiz kodlar yazmaktır.

Bu örnekteki problemi çözmek için strategy design patterni uygulayabiliriz.   
Strategy design pattern bir amaca ulaşmak için farklı stratejileri kullanıma hazır hale getiriyoruz. 

Basit bir örnekle sistemi anlayalım ve sonrasında problemimize geri döneceğiz.

Farklı indirim stratejilerini alabilen bir item sınıfı oluşturark başlayalım

In [1]:
""" Uygulamamız içi özel bir item"""
class Item:

    """Fiyat ve indirim methodunu alan bir init """

    def __init__(self, price, discount_strategy = None):
        self.price = price
        self.discount_strategy = discount_strategy
        
    """ init sırasında verilen stratejiye göre fiyat ayarlayan method"""
    def price_after_discount(self):
        
        if self.discount_strategy:
            discount = self.discount_strategy(self)
        else:
            discount = 0
            
        return self.price - discount
#dunder method ile özel print yapısı
    def __repr__(self):
        
        statement = "Price: {}, price after discount: {}"
        return statement.format(self.price, self.price_after_discount())

artık stratejilere ihtiyacımız var, hadi bunları yazalım

In [2]:


""" yüzde yirmi beş indirim yapan ve üzerine birde yirmi  indirim eklesin """
def on_sale_discount(order):
    
    return order.price * 0.25 + 20

"""  sadece yüzde yirmi indirim """
def twenty_percent_discount(order):
    
    return order.price * 0.20




artk ugulanabilir

In [4]:
test_item=Item(2000) # fiyat var ama strateji none

print(test_item)

Price: 2000, price after discount: 2000


In [5]:
test_item.discount_strategy = twenty_percent_discount
print(test_item)

Price: 2000, price after discount: 1600.0


In [6]:
test_item.discount_strategy = on_sale_discount
print(test_item)

Price: 2000, price after discount: 1480.0


Artık objemiz için bir strateji seçebiliyoruz ve objemiz buna göre davranış segiliyor

Şimdi filitreleme problemine dönelim

Tarif objeleri ve detaylarını oluşturduk bunlarıda tarif databaseimize ekledik

In [26]:
class Recepie:
    def __init__(self, ingredients=None, time=0, requires=None,name="recepie"):
        self.ingredients = ingredients
        self.time = time
        self.requires = requires
        self.name=name

    def __str__(self):  ## print ile kullanınca çıkan bu 
        return self.name
    def __repr__(self):  ## print ve repr ile ya da list içi printte çıkan bu 
        return self.name
recepie1 = Recepie(ingredients=["ing1","ing2"],time=10,requires=["steel wok"],name="Receipe1")
recepie2 = Recepie(ingredients=["ing2","ing3"],time=20,requires=["cast iron pan"],name="Receipe2")
recepie3 = Recepie(ingredients=["ing1","ing3"],time=15,requires=None ,name="Receipe3" )
recepie4 = Recepie(ingredients=["ing3","ing2"],time=20,requires=None ,name="Receipe4")
recepie5 = Recepie(ingredients=["ing1","ing2","ing3"],time=10,requires=["steel wok"] ,name="Receipe5")

recepiesDataBase=[recepie1,recepie2,recepie3,recepie4,recepie5]



In [44]:
def ingredientFilter(availableIngredients:list, receipesList:list) -> list:
    filteredList=[]
    for i in receipesList:
        if all(a  in availableIngredients for a in i.ingredients):   ### list comprehension ve all fonksiyonu ile pythonic bir yaklaşım.
            filteredList.append(i)
    return filteredList


### list comprehension ve all fonksiyonu ile pythonic bir yaklaşım.
### Eğer aşina değilseniz bastiçe : list comprehension listenizi bir ifade olarak yamzayı sağlar 
### a in available ingredients kısmı listenin elamnları olacak true yada false 
### bu listeyi for ile yapılan bir iterasyon ile oluşturduk  for a in i.ingredients
### bu alttaki ifadeye benzer
### 
### list=[]
### for a in i.ingredients:
###     list.append(a  in availableIngredients)
### all() basitçe eğer içi hep true ise true döndürür bir false bile durumu bozar



lets test our filter

In [45]:
print( ingredientFilter(availableIngredients=["ing1","ing2","ing3"], receipesList=recepiesDataBase))

[Receipe1, Receipe2, Receipe3, Receipe4, Receipe5]


In [46]:
print( ingredientFilter(availableIngredients=["ing2","ing3"], receipesList=recepiesDataBase))

[Receipe2, Receipe4]


şimdi malzemeler ve süre için de filitreleyen stratejiler yazalım

In [53]:
def itemFilter(availableItems:list, receipesList:list) -> list:
    filteredList=[]
    for i in receipesList:
        if i.requires is None or  all(a  in availableItems for a in i.requires):  ###eğer requires none ise de if  çalışır 
            filteredList.append(i)
    return filteredList

In [54]:
print( itemFilter(availableItems=["steel wok"], receipesList=recepiesDataBase))

[Receipe1, Receipe3, Receipe4, Receipe5]


In [55]:
def timeFilter(availableTime:int, receipesList:list) -> list:
    filteredList=[]
    for i in receipesList:
        if availableTime >= i.time:
            filteredList.append(i)
    return filteredList

In [57]:
print( timeFilter(availableTime=10, receipesList=recepiesDataBase))

[Receipe1, Receipe5]


Artık stratejileri uygfulayacak bi listeleme sistemine ihtiyacımız var